In [50]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [51]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [52]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [53]:
# Use the citipy module to determine city based on latitude and longitude.
#  6.1.4
from citipy import citipy

In [54]:
# Create a list for holding the cities. 6.1.5
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

605

In [55]:
# 6.2.2
import requests
requests.__version__

'2.25.1'

In [56]:
# Import the API key. 6.2.3
from configW import weather_api_key

In [57]:
# Starting URL for Weather Map API Call. 6.2.3
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0546ad5dbee13bad86394b7d864b452c


In [58]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [59]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters. We want the first iteration of the logging for each recorded response and the set to start at 1
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [60]:
# Loop through all the cities in the list 6.2.6.
for i, city in enumerate(cities):

        # Group cities in sets of 50 for logging purposes.(while grouping our records in sets of 50)
    if (i % 50 == 0 and i >= 50):
        #If the statement is true, then the set_count and the record_count are incremented by 1
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    #without enumerate() it was: city_url = url + "&q=" + cities[i]
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    #removing the blank spaces in the city name 
    # and concatenating the city name. This will find the corresponding weather data for the city 
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    #instead of finding the weather data for the first part of the city name
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.

    try:
        # Parse the JSON and retrieve data.
            #If there is no weather data for the city, i.e. , a <Response [404]> 
            #then there is no weather to retrieve and City not found. Skipping... is printed.
            # If there is weather data for the city, we will retrieve the latitude, longitude, 
            #maximum temperature, humidity, cloudiness, wind speed, and date and assign those values to variables
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        #We append the cities list with a dictionary for that city, 
        #where the key-value pairs are the values from our weather parameter
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        #Generally, it isn't good coding practice to add the pass statement to the except block. 
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | puerto ayora
Processing Record 2 of Set 1 | vaini
Processing Record 3 of Set 1 | upernavik
Processing Record 4 of Set 1 | bosaso
Processing Record 5 of Set 1 | hilo
Processing Record 6 of Set 1 | santa isabel do rio negro
Processing Record 7 of Set 1 | clyde river
Processing Record 8 of Set 1 | deniliquin
Processing Record 9 of Set 1 | bengkulu
Processing Record 10 of Set 1 | port alfred
Processing Record 11 of Set 1 | atbasar
Processing Record 12 of Set 1 | hermanus
Processing Record 13 of Set 1 | lorengau
Processing Record 14 of Set 1 | cabo san lucas
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | hobart
Processing Record 17 of Set 1 | punta arenas
Processing Record 18 of Set 1 | mar del plata
Processing Record 19 of Set 1 | faanui
Processing Record 20 of Set 1 | yellowknife
Processing Record 21 of Set 1 | cascais
Processing Record 22 of Set 1 | mae sai
Processing Record 23 of Set 1 | beringovskiy
Processing Record 24 of Set 1 |

Processing Record 41 of Set 4 | campestre
Processing Record 42 of Set 4 | simitli
Processing Record 43 of Set 4 | cabedelo
Processing Record 44 of Set 4 | attawapiskat
City not found. Skipping...
Processing Record 45 of Set 4 | tuatapere
Processing Record 46 of Set 4 | klaksvik
Processing Record 47 of Set 4 | mys shmidta
City not found. Skipping...
Processing Record 48 of Set 4 | constitucion
Processing Record 49 of Set 4 | tura
Processing Record 50 of Set 4 | tacoronte
Processing Record 1 of Set 5 | jacareacanga
Processing Record 2 of Set 5 | mount gambier
Processing Record 3 of Set 5 | parvatipuram
Processing Record 4 of Set 5 | passau
Processing Record 5 of Set 5 | inongo
Processing Record 6 of Set 5 | saint anthony
Processing Record 7 of Set 5 | harer
Processing Record 8 of Set 5 | bilma
Processing Record 9 of Set 5 | richards bay
Processing Record 10 of Set 5 | nha trang
Processing Record 11 of Set 5 | bam
Processing Record 12 of Set 5 | chagda
City not found. Skipping...
Processi

Processing Record 30 of Set 8 | simao
Processing Record 31 of Set 8 | barhiya
Processing Record 32 of Set 8 | westport
Processing Record 33 of Set 8 | yuryev-polskiy
Processing Record 34 of Set 8 | san quintin
Processing Record 35 of Set 8 | jvari
Processing Record 36 of Set 8 | valparaiso
Processing Record 37 of Set 8 | novyy urengoy
Processing Record 38 of Set 8 | berlevag
Processing Record 39 of Set 8 | mayor pablo lagerenza
Processing Record 40 of Set 8 | sistranda
Processing Record 41 of Set 8 | hailar
Processing Record 42 of Set 8 | karawang
Processing Record 43 of Set 8 | isangel
Processing Record 44 of Set 8 | ostersund
Processing Record 45 of Set 8 | carlsbad
Processing Record 46 of Set 8 | vaitupu
City not found. Skipping...
Processing Record 47 of Set 8 | amahai
Processing Record 48 of Set 8 | auki
Processing Record 49 of Set 8 | sao domingos do maranhao
Processing Record 50 of Set 8 | escanaba
Processing Record 1 of Set 9 | saint-joseph
Processing Record 2 of Set 9 | tres p

Processing Record 18 of Set 12 | qostanay
Processing Record 19 of Set 12 | iralaya
Processing Record 20 of Set 12 | mbuyapey
Processing Record 21 of Set 12 | galiwinku
City not found. Skipping...
Processing Record 22 of Set 12 | gigmoto
Processing Record 23 of Set 12 | komsomolskiy
Processing Record 24 of Set 12 | aranos
Processing Record 25 of Set 12 | nalut
Processing Record 26 of Set 12 | zubtsov
Processing Record 27 of Set 12 | leningradskiy
Processing Record 28 of Set 12 | prienai
Processing Record 29 of Set 12 | taburi
City not found. Skipping...
Processing Record 30 of Set 12 | nakhon si thammarat
Processing Record 31 of Set 12 | goderich
Processing Record 32 of Set 12 | am timan
Processing Record 33 of Set 12 | mandera
Processing Record 34 of Set 12 | dole
Processing Record 35 of Set 12 | mosteni
City not found. Skipping...
Processing Record 36 of Set 12 | inverell
Processing Record 37 of Set 12 | fort saint john
City not found. Skipping...
Processing Record 38 of Set 12 | kysy

In [61]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.7393,-90.3518,78.76,80,86,11.92,EC,2021-08-02 17:17:10
1,Vaini,-21.2000,-175.2000,68.16,77,20,11.50,TO,2021-08-02 17:15:20
2,Upernavik,72.7868,-56.1549,46.11,66,1,5.55,GL,2021-08-02 17:17:10
3,Bosaso,11.2842,49.1816,92.28,45,16,12.88,SO,2021-08-02 17:17:10
4,Hilo,19.7297,-155.0900,73.13,84,90,5.75,US,2021-08-02 17:13:01


In [65]:
# Reorder the columns and run the cell
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(5)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Puerto Ayora,EC,2021-08-02 17:17:10,-0.7393,-90.3518,78.76,80,86,11.92
1,Vaini,TO,2021-08-02 17:15:20,-21.2000,-175.2000,68.16,77,20,11.50
2,Upernavik,GL,2021-08-02 17:17:10,72.7868,-56.1549,46.11,66,1,5.55
3,Bosaso,SO,2021-08-02 17:17:10,11.2842,49.1816,92.28,45,16,12.88
4,Hilo,US,2021-08-02 17:13:01,19.7297,-155.0900,73.13,84,90,5.75


In [63]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [64]:
# Import linregress
from scipy.stats import linregress

# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):

    # Run regression on hemisphere weather data.
    (slope, intercept, r_value, p_value, std_err) = linregress(x_values, y_values)

    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()